In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import numpy as np
np.random.seed(5)
import pandas as pd
import time

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
import sys
sys.path.insert(1, '/cluster/home/omineeva/ResMiCo')
from resmico import contig_reader

In [20]:
project_path = '/cluster/home/omineeva/global_projects/projects/projects2019-contig_quality/'

In [21]:
def get_ind(reader, sel_file_name='', sel_label=2, fraq=1):
    #sel_label: 0 - negative, 1 - positive, 2 - both
    sel_ind = []
    for i in range(len(reader)):
        add = True
        if sel_label==0:
            if reader.contigs[i].misassembly != 0:
                add = False
        elif sel_label==1:
            if reader.contigs[i].misassembly == 0:
                add = False 
        if sel_file_name not in reader.contigs[i].file:
            add = False
        if add:
            sel_ind.append(i)
    if fraq < 1:
        num = int(fraq * len(sel_ind))
        np.random.shuffle(sel_ind)
        sel_ind = sel_ind[:num] 
    return sel_ind



def compute_mean_stdev(reader, sel_file_name='', sel_label=2, fraq=1):
    start_time = time.time()
    sel_ind = get_ind(reader, sel_file_name, sel_label, fraq)
    print('sel_ind func time: ',  time.time()-start_time, len(sel_ind))
    
    contig_data = [reader.contigs[i] for i in sel_ind]
    
    start_time = time.time()
    features_data = reader.read_contigs(contig_data, return_raw=True)
    print('read_contigs func time: ',  time.time()-start_time)
    
    features_stats = {}
    for feature in reader.feature_names:
        features_stats[feature] = {'sum':0, 'sum2':0, 'count':0}

    start_time = time.time()
    
    #for runs with 1 feature
    minval = 10**8
    maxval = -10**8
    for contig in features_data:
        for feature in reader.feature_names:
            cur_max = np.nanmax(contig[feature])
            cur_min = np.nanmin(contig[feature])
            minval = np.min((minval,cur_min))
            maxval = np.max((maxval,cur_max))
            
            features_stats[feature]['sum'] += np.nansum(contig[feature])
            features_stats[feature]['sum2'] += np.nansum(contig[feature]**2)
            features_stats[feature]['count'] += (len(contig[feature]) -
                                             np.count_nonzero(np.isnan(contig[feature])))
    print('iterate features_data time: ',  time.time()-start_time)
    print(minval, maxval)
    print(features_stats)
    features_meanstd = {}
    for feature in reader.feature_names:
        feat_sum = features_stats[feature]['sum']
        n_el = features_stats[feature]['count']
        feat_sq_sum = features_stats[feature]['sum2']
        features_meanstd[feature+'_mean'] = feat_sum / n_el
        features_meanstd[feature+'_stdev'] = np.sqrt((feat_sq_sum / n_el - 
                                                     (feat_sum / n_el) ** 2).clip(min=0))
    return features_meanstd

In [22]:
# features = ['num_SNPs', 'num_proper_Match', 'num_orphans_Match', 
#             'mean_al_score_Match', 'coverage', 'stdev_insert_size_Match', 'mean_mapq_Match']

features=['coverage']#'num_proper_Match',
#         'num_orphans_Match',
#         'max_insert_size_Match',
#         'mean_insert_size_Match',
#         'min_insert_size_Match',
#         'stdev_insert_size_Match',
#         'mean_mapq_Match',
#         'min_mapq_Match',
#         'stdev_mapq_Match',
#         'mean_al_score_Match',
#         'min_al_score_Match',
#         'stdev_al_score_Match',
#         'seq_window_perc_gc',
#         'num_proper_SNP',
#         'coverage']#,  
#           'seq_window_entropy']

In [23]:
df = pd.DataFrame(columns=np.concatenate((('data_path', 'sel_file_name', 'sel_label'), 
                                          [f+'_mean' for f in features],
                                          [f+'_stdev' for f in features])))

In [24]:
row_ind = 0
n_procs = 16
chunks = False
no_cython = False

## TRAIN

In [25]:
feature_files_path = project_path + 'data/v2/resmico-sm/GTDBr202_n9k_train/' #_1rep
reader = contig_reader.ContigReader(feature_files_path, features, n_procs, chunks, no_cython)
print(len(reader))

8637153


In [11]:
sum_a = 0
sum_l = 0
for i, cont in enumerate(reader.contigs):
    sum_l += cont.length
    sum_a += cont.avg_coverage*cont.length
sum_a/sum_l #if >20 we can not trust it, there are dummy 100s

53.81759907545678

In [12]:
print(sum_l)

76149223294


In [13]:
import statistics
sum_l = []
for i, cont in enumerate(reader.contigs):
    sum_l.append(cont.length)
statistics.median(sum_l), np.mean(sum_l)

(1517.0, 2959.456903766341)

In [14]:
ms = 0
for i, cont in enumerate(reader.contigs):
    ms+=1*(cont.misassembly>0)
ms/(i+1)

0.03176071798750214

In [15]:
sel_file_name = ''
sel_label=2
fraq=0.1

features_meanstd = compute_mean_stdev(reader, sel_file_name, sel_label, fraq)

features_meanstd['data_path'] = feature_files_path[len(project_path):]
features_meanstd['sel_file_name'] = sel_file_name
features_meanstd['sel_label'] = sel_label
row_to_add = pd.Series(features_meanstd, name=row_ind)
row_ind+=1
df = df.append(row_to_add)

display(df)

sel_ind func time:  10.679700136184692 2573081
read_contigs func time:  3413.3425109386444
iterate features_data time:  214.11487698554993
0.0 1264.0
{'coverage': {'sum': 95697636517.0, 'sum2': 1708121085599.0, 'count': 7603137259}}


,data_path,sel_file_name,sel_label,coverage_mean,coverage_stdev
0,data/v2/resmico-sm/GTDBr202_n9k_train/,,2,12.586599,8.138647


In [50]:
feature_files_path = project_path + 'data/CAMI/CAMI2_HMP-skin/short_read/ResMiCo-SM/features'
reader = contig_reader.ContigReader(feature_files_path, features, n_procs, chunks, no_cython)

sel_file_name = ''#'0.005555/mean-10-sigma-1/3/150/8000000'
sel_label=2
fraq=1

features_meanstd = compute_mean_stdev(reader, sel_file_name, sel_label, fraq)
features_meanstd['data_path'] = feature_files_path[len(project_path):]
features_meanstd['sel_file_name'] = sel_file_name
features_meanstd['sel_label'] = sel_label
row_to_add = pd.Series(features_meanstd, name=row_ind)
row_ind+=1
df = df.append(row_to_add)

display(df)

sel_ind func time:  0.06417536735534668 323396
read_contigs func time:  65.99628233909607
iterate features_data time:  22.14345407485962
0.0 1004.0
{'coverage': {'sum': 13979831488.0, 'sum2': 273458914943.0, 'count': 890427979}}


,data_path,sel_file_name,sel_label,coverage_mean,coverage_stdev
0,data/CAMI/CAMI2_HMP-oral/short_read/ResMiCo-SM...,,2,16.330370,7.811221
1,data/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/...,,2,16.900280,7.235160
2,data/CAMI/CAMI2_HMP-skin/short_read/ResMiCo-SM...,,2,15.700126,7.785600
